In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from copy import deepcopy

In [2]:
st = pd.HDFStore(os.path.expanduser("/Users/nipunbatra/wiki-all.h5"))

In [3]:
metadata_df = pd.read_csv("/Users/nipunbatra/w/dataport-metadata.csv",index_col=0)

In [4]:
months=3
start='7-1-2014'
start_month=7
end = '9-30-2014'
end_month=10

In [66]:
sd_df = pd.read_csv("../data/input/san_diego_df.csv", index_col=0)
unscaled_sd_df = pd.read_csv("../data/input/unscaled_san_diego_df.csv", index_col=0)

bo_df = pd.read_csv("../data/input/boulder_df.csv", index_col=0)
unscaled_bo_df = pd.read_csv("../data/input/unscaled_boulder_df.csv", index_col=0)

In [6]:
import json
sd_homes = json.load(open("../data/input/san_diego_homes.json",'r'))
bo_homes = json.load(open("../data/input/boulder_homes.json",'r'))

In [22]:
national_average = {}
national_average["hvac"] = 13

regional_average = {}
regional_average["sd"] = {}
regional_average["sd"]["hvac"] = 18

In [46]:
def _find_accuracy_na(home, appliance):
    if appliance =="hvac":
        start, stop=7,10
    else:
        start, stop=7, 10

    
    pred_test = {}
    gt_test = {}
    for month in range(start, stop):
        
        pred_test[month] = unscaled_df.ix[home]["aggregate_%d" %month]*national_average[appliance]/100
        gt_test[month] = unscaled_df.ix[home]['%s_%d' %(appliance, month)]

    #print {'f':F_best, 'k':K_best,'accuracy':accur_max}
    #json.dump({'f':F_best, 'k':K_best,'accuracy':accur_max},open("../main-out-new/%s_%s_%d.json" %(appliance,feature, home),"w") )
    
    
    pred_df = pd.Series(pred_test)
   
    
    gt_df = pd.Series(gt_test)
    error = (gt_df-pred_df).abs().div(gt_df).mul(100)
    accuracy_test = 100-error
    accuracy_test[accuracy_test<0]=0

    return pred_df, gt_df, accuracy_test.squeeze()

In [59]:
_find_accuracy_na(2523, "hvac")

(7    195.820800
 8    154.969601
 9    135.541536
 dtype: float64, 7    135.648438
 8     70.280182
 9     15.237200
 dtype: float64, 7    55.640947
 8     0.000000
 9     0.000000
 dtype: float64)

In [75]:
acc = {}
SD = False
if SD:
    df, unscaled_df = sd_df, unscaled_sd_df
    homes = sd_homes
else:
    df, unscaled_df = bo_df, unscaled_bo_df
    homes = bo_homes
    


for appliance in ["hvac"]:
    
    acc[appliance] = {}
    feature="Monthly"
    for home in homes[appliance][:]:
        try:
            pred, gt, out_df = _find_accuracy_na(home, appliance)
            
            acc[appliance][home]=out_df
            print home, acc[appliance][home].mean()
        except Exception, e:
            print e

232 40.7300165583
698 31.73929822
1879 37.0017164541
2144 29.4964556473
2523 18.5469823884
2903 81.6500856334
3734 44.8399897874
4147 63.7167454027
4920 51.345750228
4974 48.953804797
5652 53.7258422814
5904 28.1338070607
7001 62.4663392502
8273 57.0659674049
8368 41.6616812373
8395 19.6438385654
9156 32.7058628676
9341 63.2406730193
9818 56.2211810139


In [76]:
pd.DataFrame(acc['hvac']).T.mean()

7    50.023981
8    54.581756
9    31.639427
dtype: float64

In [12]:
#for appliance in ["hvac","fridge","dw","mw"]:
for appliance in ["fridge","hvac"]:
    if appliance in acc:
        continue
    else:
        print appliance
        acc[appliance] = {}
        feature="Monthly"
        for home in bo_homes[appliance][:]:
            try:
                pred, gt, out_df = _find_accuracy(home, appliance, feature)
                acc[appliance][home]=out_df
                print home, acc[appliance][home].mean()
            except Exeption, e:
                print e

fridge
698 96.5397271845
1350 67.8035880339
1879 58.7388484772
2144 97.0705590542
2523 90.3224642219
2903 55.129893089
3104 75.5938205903
3353 41.6497319685
3734 70.7178716102
4601 86.5695956932
4703 60.0667680629
4830 83.6099476131
4920 63.6088584776
4974 63.3826177532
5652 84.7860542967
5904 60.0681232139
6148 57.9375172364
6264 27.893513418
6888 89.4637942679
6956 58.2080819012
7818 44.3389132385
8273 55.8005155956
8368 64.5161186165
8622 83.3254787276
8729 36.7405109653
8862 77.1903734155
9085 78.7771532917
9341 94.0508955593
9818 91.7869015655
hvac
232 91.8903077922
698 0.0
1879 23.9480543358
2144 67.6237720249
2523 0.0
2903 0.0
3734 56.225860573
4147 63.6444349687
4920 0.0
4974 87.5829012459
5652 0.0
5904 90.821901653
7001 52.6623072259
8273 4.02898525246
8368 81.7060763746
8395 50.1667861585
9156 60.7619690893
9341 91.9657155
9818 94.5015282017


In [42]:
pd.Series(acc['hvac']).mean()

46.353887346772474

In [43]:
mean_across_months = {}
for appliance in acc.keys():
    if appliance =="hvac":
        mean_across_months[appliance] = pd.Series(acc[appliance])
    else:
        mean_across_months[appliance] = pd.DataFrame(acc[appliance])
    


mean_across_months['fridge'].loc[10, 2144] = np.NaN
mean_across_months['fridge'].loc[7, 7818] = np.NaN#mean_across_months['hvac'].loc[9, 3938] = np.NaN

for appliance in mean_across_months.keys():
    if appliance!="hvac":
        mean_across_months[appliance] = mean_across_months[appliance].mean()

In [44]:
pd.DataFrame(acc['fridge']).mean().mean()

68.50575613455632

In [45]:
pd.DataFrame(mean_across_months).mean()

fridge    69.474575
hvac      46.353887
dtype: float64